
<h3 style="text-align: center;"><b>На основе ШАД Яндекс</b></h3><h1 id="Домашнее-задание.-Обучение-нейронных-сетей-на-PyTorch.">Домашнее задание. Обучение нейронных сетей на PyTorch.<a class="anchor-link" href="#Домашнее-задание.-Обучение-нейронных-сетей-на-PyTorch.">¶</a></h1><p>В этом домашнем задании вам предстоит предсказывать типы небесных объектов. Эту задачу вы будете решать с помощью нейронных сетей, используя библиотеку PyTorch.</p>
<p>Вам необходимо заполнить пропуски в ноутбуке. Кое-где вас просят сделать выводы о проделанной работе. Постарайтесь ответить на вопросы обдуманно и развёрнуто.</p>


In [ ]:

import torch
from torch import nn
from torch import functional as F
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt




<h1 id="Дисклеймер-про-CrossEntropyLoss-и-NLLLoss">Дисклеймер про CrossEntropyLoss и NLLLoss<a class="anchor-link" href="#Дисклеймер-про-CrossEntropyLoss-и-NLLLoss">¶</a></h1><p>Обычно в PyTorch не нужно делать Softmax как последний слой модели.</p>
<ul>
<li>Если Вы используете NLLLoss, то ему на вход надо давать лог вероятности, то есть выход слоя LogSoftmax. (Просто результат софтмакса, к которому применен логарифм)</li>
<li>Если Вы используете CrossEntropyLoss, то применение LogSoftmax уже включено внутрь лосса, поэтому ему на вход надо подавать просто выход обычного линейного слоя без активации. По сути CrossEntropyLoss = LogSoftmax + NLLLoss</li>
</ul>
<p>Зачем такие сложности, чтобы посчитать обычную кросс энтропию? Дело в том, что нам в любом случае придется взять логарифм от результатов софтмакса, а если делать это одной функцией, то можно сделать более устойчивую реализацию, которая даст меньшую вычислительную погрешность.</p>
<p>Таким образом, если у вас в конце сети, решающей задачу классификации, стоит просто линейный слой без активации, то вам нужно использовать CrossEntropy. В этой домашке везде используется лосс CrossEntropy</p>



<h1 id="Задание-1.-Создайте-генератор-батчей.">Задание 1. Создайте генератор батчей.<a class="anchor-link" href="#Задание-1.-Создайте-генератор-батчей.">¶</a></h1><p>В этот раз мы хотим сделать генератор, который будет максимально похож на то, что используется в реальном обучении.</p>
<p>С помощью numpy вам нужно перемешать исходную выборку и выбирать из нее батчи размером batch_size, если размер выборки не делился на размер батча, то последний батч должен иметь размер меньше batch_size и состоять просто из всех оставшихся объектов. Возвращать нужно в формате (X_batch, y_batch). Необходимо написать именно генератор, то есть вместо return использовать yield.</p>
<p>Хорошая статья про генераторы: <a href="https://habr.com/ru/post/132554/">https://habr.com/ru/post/132554/</a></p>
<p><strong>Ответ на задание - код</strong></p>


In [ ]:

def batch_generator(X, y, batch_size):
    np.random.seed(42)
    perm = np.random.permutation(len(X))
    
    # YOUR CODE




<p>Попробуем потестировать наш код</p>


In [ ]:

from inspect import isgeneratorfunction
assert isgeneratorfunction(batch_generator), "batch_generator должен быть генератором! В условии есть ссылка на доки"

X = np.array([
              [1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]
])
y = np.array([
              1, 2, 3
])

# Проверим shape первого батча
iterator = batch_generator(X, y, 2)
X_batch, y_batch = next(iterator)
assert X_batch.shape == (2, 3), y_batch.shape == (2,)
assert np.allclose(X_batch, X[:2]), np.allclose(y_batch, y[:2])

# Проверим shape последнего батча (их всего два)
X_batch, y_batch = next(iterator)
assert X_batch.shape == (1, 3), y_batch.shape == (1,)
assert np.allclose(X_batch, X[2:]), np.allclose(y_batch, y[2:])

# Проверим, что итерации закончились
iter_ended = False
try:
    next(iterator)
except StopIteration:
    iter_ended = True
assert iter_ended

# Еще раз проверим то, сколько батчей создает итератор
X = np.random.randint(0, 100, size=(1000, 100))
y = np.random.randint(-1, 1, size=(1000, 1))
num_iter = 0
for _ in batch_generator(X, y, 3):
    num_iter += 1
assert num_iter == (1000 // 3 + 1)




<h1 id="Задание-2.-Обучите-модель-для-классификации-звезд">Задание 2. Обучите модель для классификации звезд<a class="anchor-link" href="#Задание-2.-Обучите-модель-для-классификации-звезд">¶</a></h1><p>Загрузите датасет из файла sky_data.csv, разделите его на train/test и обучите на нем нейронную сеть (архитектура ниже). Обучайте на батчах с помощью оптимизатора Adam, lr подберите сами, пробуйте что-то вроде 1e-2</p>
<p>Архитектура:</p>
<ol>
<li>Dense Layer с relu активацией и 50 нейронами</li>
<li>Dropout 80% (если другой keep rate дает сходимость лучше, то можно изменить) (попробуйте 50%) </li>
<li>BatchNorm</li>
<li>Dense Layer с relu активацией и 100 нейронами</li>
<li>Dropout 80% (если другой keep rate дает сходимость лучше, то можно изменить) (попробуйте для разнообразия 50%)</li>
<li>BatchNorm</li>
<li>Выходной Dense слой c количеством нейронов, равному количеству классов</li>
</ol>
<p>Лосс - CrossEntropy.</p>



<p>В датасете классы представлены строками, поэтому классы нужно закодировать. Для этого в строчке ниже объявлен dict, с помощью него и функции map превратите столбец с таргетом в целое число. Кроме того, за вас мы выделили признаки, которые нужно использовать.</p>



<h3 id="Загрузка-и-обработка-данных">Загрузка и обработка данных<a class="anchor-link" href="#Загрузка-и-обработка-данных">¶</a></h3>


In [ ]:

feature_columns = ['ra', 'dec', 'u', 'g', 'r', 'i', 'z', 'run', 'camcol', 'field']
target_column = 'class'

target_mapping = {
    'GALAXY': 0,
    'STAR': 1,
    'QSO': 2
}



In [ ]:

data = pd.read_csv('https://drive.google.com/uc?id=1K-8CtATw6Sv7k2dXco1fL5MAhTbKtIH3')
data['class'].value_counts()



In [ ]:

data.head()



In [ ]:

# Extract Features
X = <YOUR CODE>
# Extract target
y = <YOUR CODE>

# encode target with target_mapping
y = <YOUR CODE>




<p>Нормализация фичей</p>


In [ ]:

# Просто вычтите среднее и поделитe на стандартное отклонение (с помощью пандас). Также преобразуйте всё в np.array
X = <YOUR CODE>



In [ ]:

assert type(X) == np.ndarray and type(y) == np.ndarray, 'Проверьте, что получившиеся массивы являются np.ndarray'
assert np.allclose(y[:5], [1,1,0,1,1])
assert X.shape == (10000, 10)
assert np.allclose(X.mean(axis=0), np.zeros(10)) and np.allclose(X.std(axis=0), np.ones(10)), 'Данные не отнормированы'




<p>Обучение</p>


In [ ]:

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
# Превратим данные в тензоры, чтобы потом было удобнее
X_train = torch.FloatTensor(X_train)
y_train = torch.LongTensor(y_train)
X_test = torch.FloatTensor(X_test)
y_test = torch.LongTensor(y_test)




<p>Хорошо, данные мы подготовили, теперь надо объявить модель</p>


In [ ]:

torch.manual_seed(42) 
np.random.seed(42)
model = nn.Sequential(
    <YOUR CODE>
)
    
loss_fn = <YOUR CODE>
optimizer = <YOUR CODE>




<h3 id="Обучающий-цикл">Обучающий цикл<a class="anchor-link" href="#Обучающий-цикл">¶</a></h3>


In [ ]:

def train(X_train, y_train, X_test, y_test, num_epoch):
    train_losses = []
    test_losses = []
    for i in range(num_epoch):
        epoch_train_losses = []
        for X_batch, y_batch in batch_generator(X_train, y_train, 500):
            # На лекции мы рассказывали, что дропаут работает по-разному во время обучения и реального предсказания
            # Чтобы это учесть нам нужно включать и выключать режим обучения, делается это командой ниже
            model.train(True)
            # Посчитаем предсказание и лосс
            # YOUR CODE
            
            # зануляем градиент
            # YOUR CODE
            
            # backward
            # YOUR CODE
            
            # ОБНОВЛЯЕМ веса
            # YOUR CODE
            
            # Запишем число (не тензор) в наши батчевые лоссы
            epoch_train_losses.append(# YOUR CODE)        
        train_losses.append(np.mean(epoch_train_losses))
        
        # Теперь посчитаем лосс на тесте
        model.train(False)
        with torch.no_grad():
            # Сюда опять же надо положить именно число равное лоссу на всем тест датасете
            test_losses.append(# YOUR CODE)
            
    return train_losses, test_losses



In [ ]:

def check_loss_decreased():
    print("На графике сверху, точно есть сходимость? Точно-точно? [Да/Нет]")
    s = input()
    if s.lower() == 'да':
        print("Хорошо!")
    else:
        raise RuntimeError("Можно уменьшить дропаут, уменьшить lr, поправить архитектуру, etc")



In [ ]:

train_losses, test_losses = train(# YOUR CODE) #Подберите количество эпох так, чтобы график loss сходился
plt.plot(range(len(train_losses)), train_losses, label='train')
plt.plot(range(len(test_losses)), test_losses, label='test')
plt.legend()
plt.show()
    
check_loss_decreased()
assert train_losses[-1] < 0.3 and test_losses[-1] < 0.3




<h3 id="Вычислите-accuracy-получившейся-модели-на-train-и-test">Вычислите accuracy получившейся модели на train и test<a class="anchor-link" href="#Вычислите-accuracy-получившейся-модели-на-train-и-test">¶</a></h3>


In [ ]:

from sklearn.metrics import accuracy_score

model.eval()
train_pred_labels = #YOUR CODE: use forward
test_pred_labels = #YOUR CODE: use forward

train_acc = accuracy_score(# YOUR CODE)
test_acc = accuracy_score(# YOUR CODE)

assert train_acc > 0.9, "Если уж классифицировать звезды, которые уже видел, то не хуже, чем в 90% случаев"
assert test_acc > 0.9, "Новые звезды тоже надо классифицировать хотя бы в 90% случаев"

print("Train accuracy: {}\nTest accuracy: {}".format(train_acc, test_acc))




<h1 id="Задание-3.-Исправление-ошибок-в-архитектуре">Задание 3. Исправление ошибок в архитектуре<a class="anchor-link" href="#Задание-3.-Исправление-ошибок-в-архитектуре">¶</a></h1><p>Только что вы обучили полносвязную нейронную сеть. Теперь вам предстоит проанализировать архитектуру нейронной сети ниже, исправить в ней ошибки и  обучить её с помощью той же функции train. Пример исправления ошибок есть в семинаре Григория Лелейтнера.</p>
<p>Будьте осторожнее и убедитесь, что перед запуском train вы вновь переопределили все необходимые внешние переменные (train обращается к глобальным переменным, в целом так делать не стоит, но сейчас это было оправдано, так как иначе нам пришлось бы передавать порядка 7-8 аргументов).</p>
<p>Чтобы у вас получилась такая же архитектура, как у нас, и ответы совпали, давайте определим некоторые правила, как исправлять ошибки:</p>
<ol>
<li>Если вы видите лишний нелинейный слой, который стоит не на своем месте, просто удалите его. (не нужно добавлять новые слои, чтобы сделать постановку изначального слоя разумной. Удалять надо самый последний слой, который все портит. Для линейных слоев надо что-то исправить, а не удалить его)</li>
<li>Если у слоя нет активации, то добавьте ReLU или другую подходящую активацию</li>
<li>Если что-то не так с learning_rate, то поставьте 1e-2</li>
<li>Если что-то не так с параметрами, считайте первый параметр, который появляется, как верный (т.е. далее в сети должен использоваться он).</li>
<li>Ошибки могут быть и в полносвязных слоях. </li>
<li>Любые другие проблемы решаются более менее однозначно, если же у вас есть серьезные сомнения, то напишите в беседу в телеграме и пинганите меня @AlexPak</li>
</ol>



<p>Задача все та же - классификация небесных объектов на том же датасете. После исправления сети вам нужно обучить ее.</p>
<p><strong>Ответ на задачу - средний лосс на тестовом датасете</strong></p>


In [ ]:

torch.manual_seed(42)   
np.random.seed(42)
# WRONG ARCH
model = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(6, 50),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(100, 200),
    nn.Softmax(),
    nn.Linear(200, 200),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(200, 3),
    nn.Dropout(p=0.5)
)


loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters[:-2], lr=1e-100)



In [ ]:

# RIGHT ARCH
torch.manual_seed(42)   
np.random.seed(42)
model = nn.Sequential(
    <YOUR CODE>
)


loss_fn = <YOUR CODE>
optimizer = <YOUR CODE>




<h3 id="Обучите-и-протестируйте-модель-так-же,-как-вы-это-сделали-в-задаче-2.-Вычислите-accuracy.">Обучите и протестируйте модель так же, как вы это сделали в задаче 2. Вычислите accuracy.<a class="anchor-link" href="#Обучите-и-протестируйте-модель-так-же,-как-вы-это-сделали-в-задаче-2.-Вычислите-accuracy.">¶</a></h3>


In [ ]:

#YOUR CODE

train_acc = <YOUR CODE>
test_acc = <YOUR CODE>


assert train_acc > 0.9, "Если уж классифицировать звезды, которые уже видел, то не хуже, чем в 90% случаев"
assert test_acc > 0.9, "Новые звезды тоже надо классифицировать хотя бы в 90% случаев"




<h2 id="Задание-4.-Сделайте-выводы">Задание 4. Сделайте выводы<a class="anchor-link" href="#Задание-4.-Сделайте-выводы">¶</a></h2><p>Начиная с какого количества блоков минимальный лосс за время обучения увеличивается? Почему лишнее количество блоков не помогает модели?</p>
